In [ ]:
# Importing all the required libraries
import csv
import codecs
import pandas as pd                          # package to store and manipulate data
import numpy as np                           # package to store and manipulate data
import matplotlib.pyplot as plt              # plotting package
import seaborn as sns                        # plotting package 
import sklearn                               # model building package
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import re                                    # packages to clean text
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
import nltk as nlp
nltk.download('wordnet')
import string
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.colors as mcolors
import tensorflow as tf

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# loading all the dataset
df1= pd.read_csv('/troll_tweets_with_sentiment.csv', encoding = "unicode_escape")
df2= pd.read_csv('/regular_user_tweets_with_sentiment.csv', encoding = "unicode_escape")
df3= pd.read_csv('/commercial_user_tweets_with_sentiment.csv', encoding = "unicode_escape")
df4= pd.read_csv('/bots_tweets_with_sentiment.csv', encoding = "unicode_escape")

In [ ]:
df = pd.concat([df1,df2,df3,df4]) # merging all datasets

In [ ]:
df.head() # displying first five rows

,tweet_created_at,tweet_text,sentiment
0,2020-09-30 21:25:00,RT AlexaZtoA More money spent on adoration of ...,pos
1,2020-09-30 21:20:00,Drallasta httpstcoP9Tv7JD0hT,neu
2,2020-09-30 21:14:00,osbornetravel guyverhofstadt NathalieLoiseau v...,pos
3,2020-09-30 21:12:00,KatyMontgomerie Canada the home of acceptance ...,neg
4,2020-09-30 21:11:00,KatyMontgomerie Even some Canadians are horrif...,neg


In [ ]:
# encoding of sentiment label
ord_enc = OrdinalEncoder()
df["sentiment"] = ord_enc.fit_transform(df[["sentiment"]])
df[["sentiment"]].head(11)

,sentiment
0,2.0
1,1.0
2,2.0
3,0.0
4,0.0
5,0.0
6,0.0
7,1.0
8,1.0
9,1.0


In [ ]:
# keeping only required features 
dataframe = df[["tweet_text", "sentiment"]]

In [ ]:
# converting sentiment label into int from float
dataframe["sentiment"] = dataframe['sentiment'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
dataframe.head()

,tweet_text,sentiment
0,RT AlexaZtoA More money spent on adoration of ...,2
1,Drallasta httpstcoP9Tv7JD0hT,1
2,osbornetravel guyverhofstadt NathalieLoiseau v...,2
3,KatyMontgomerie Canada the home of acceptance ...,0
4,KatyMontgomerie Even some Canadians are horrif...,0


In [ ]:
seq_len = 64              # setting max sequence length
num_samples = len(df)     # checking length of entire dataset

Xids = np.zeros((num_samples, seq_len))       # creating an array of zeros for input with df length
Xmask = np.zeros((num_samples, seq_len))      # creating an array of zeros for masking with df length
Xids.shape                                    # checking the shape of the input

(69772, 64)

In [ ]:
!pip install transformers     # installing transformers 

In [ ]:
import transformers # importing transformers

In [ ]:
from transformers import BertTokenizer   # importing Bert tokenizer from the pretrained model

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')   # creating toknizer from the pre trained model

In [ ]:
# creating tokens from the pre trained tokenizer with special tokens
for i, tweet_text in enumerate(df['tweet_created_at']):
  tokens= tokenizer.encode_plus(str(tweet_text), max_length=seq_len, truncation=True, padding='max_length',
                                add_special_tokens =True, return_tensors ='tf')
  Xids[i, :] = tokens['input_ids']              # assigning tokens to the already created input array 
  Xmask[i, :] = tokens['attention_mask']        # assigning tokens to the alraedy created attention mask input

In [ ]:
Xids        # checking the shape of the input

array([[  101., 12795.,   118., ...,     0.,     0.,     0.],
       [  101., 12795.,   118., ...,     0.,     0.,     0.],
       [  101., 12795.,   118., ...,     0.,     0.,     0.],
       ...,
       [  101.,   128.,   120., ...,     0.,     0.,     0.],
       [  101.,   128.,   120., ...,     0.,     0.,     0.],
       [  101.,   128.,   120., ...,     0.,     0.,     0.]])

In [ ]:
# converting the sentiment labels to array 
arr = dataframe['sentiment'].values

In [ ]:
arr    # labels array

array([2, 1, 2, ..., 2, 2, 2])

In [ ]:
# creating labels with zeros 
labels = np.zeros((num_samples, arr.max()+1))

In [ ]:
# assigning sentiment labesl to the created zero labels for one hot encoding
labels[np.arange(num_samples), arr] =1

In [ ]:
labels       

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [ ]:
# creating input to the bert model
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))
dataset.take(1)

<TakeDataset shapes: ((64,), (64,), (3,)), types: (tf.float64, tf.float64, tf.float64)>

In [ ]:
# mapping imput, mask, labels
def map_fun(input_ids, mask, labels):
  return {"input_ids":input_ids, "attention_mask":mask}, labels

In [ ]:
dataset = dataset.map(map_fun)

In [ ]:
dataset.take(1)

<TakeDataset shapes: ({input_ids: (64,), attention_mask: (64,)}, (3,)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.float64)>

In [ ]:
batch_size =128    # creatng batches of the data for training
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder =True)  # creating batches and shuffling the data 
dataset.take(1)

<TakeDataset shapes: ({input_ids: (128, 64), attention_mask: (128, 64)}, (128, 3)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.float64)>

In [ ]:
split =0.8   # spitting the data for training and testing
size =int(((num_samples/batch_size) *split))

In [ ]:
size    # train data batches 

436

In [ ]:
train_ds =dataset.take(size)    # train dataset size 
test_ds = dataset.skip(size)    # test dataset size
del dataset

In [ ]:
from transformers import TFAutoModel   # importing bert model

In [ ]:
bert = TFAutoModel.from_pretrained('bert-base-cased')   # loading an object of pre trained bert model

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
bert.summary()     # model summary

Model: "tf_bert_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# craeting model architecture
input_ids  = tf.keras.layers.Input(shape= (seq_len,), name= 'input_ids', dtype='int32') # input for the model
mask  = tf.keras.layers.Input(shape= (seq_len,), name= 'attention_mask', dtype='int32') # attention mask
embeddings = bert.bert(input_ids, attention_mask = mask)[1]                             # embeddings from the pre trained bert model
x = tf.keras.layers.Dense(512, activation  ='relu')(embeddings)                         # input layer
y = tf.keras.layers.Dense(arr.max()+1, activation='softmax', name ='outputs')(x)        # output layer


In [ ]:
model =tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model.layers[2].trainable = False
model.summary()    # model summary with additional fine tuning layers 

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 64)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 64)]         0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 512)          393728      bert[0][1]                 

In [ ]:
# assigning model parameters 
optimizer = tf.keras.optimizers.Adam(lr =1e-5, decay =1e-6)    # model optimizer 
loss =tf.keras.losses.CategoricalCrossentropy()           # loss function of the model
acc =tf.keras.metrics.CategoricalAccuracy()               # accuracy of the model

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model.compile(optimizer=optimizer, loss =loss, metrics=[acc]) # model compilation


In [ ]:
history =model.fit(train_ds, epochs =1)  # training the model

436/436 [==============================] - 603s 1s/step - loss: 0.7214 - categorical_accuracy: 0.6820 - val_loss: 0.4465 - val_categorical_accuracy: 0.8424


In [ ]:
results = model.evaluate(test_ds)        # evaluating the model with test data

109/109 [==============================] - 105s 943ms/step - loss: 0.4438 - categorical_accuracy: 0.8437
